In [1]:
import pandas as pd
import ydata_profiling as pp
from pycaret.regression import *
pd.set_option('display.max_columns', None)

In [20]:
dataset = pd.read_csv("data_properti_filtered.csv")
dataset = dataset[dataset['province'] == 'Jawa Timur']
dataset = dataset.iloc[:35000]
dataset = dataset[dataset['bedRooms'] <= 8]
dataset.head()

,price,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,district,cluster
0,850000000,100,50,3.0,2,0,Jawa Timur,Surabaya,Tambaksari,0
1,1800000000,150,180,3.0,2,0,Jawa Timur,Sidoarjo,Sidoarjo,0
2,6000000000,350,385,6.0,4,0,Jawa Timur,Sidoarjo,Sidoarjo,0
3,550000000,70,80,2.0,2,0,Jawa Timur,Surabaya,Asemrowo,0
4,1499000000,122,88,3.0,3,0,Jawa Timur,Surabaya,Mulyorejo,0


In [21]:
dataset.head(100)

,price,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,district,cluster
0,850000000,100,50,3.0,2,0,Jawa Timur,Surabaya,Tambaksari,0
1,1800000000,150,180,3.0,2,0,Jawa Timur,Sidoarjo,Sidoarjo,0
2,6000000000,350,385,6.0,4,0,Jawa Timur,Sidoarjo,Sidoarjo,0
3,550000000,70,80,2.0,2,0,Jawa Timur,Surabaya,Asemrowo,0
4,1499000000,122,88,3.0,3,0,Jawa Timur,Surabaya,Mulyorejo,0
...,...,...,...,...,...,...,...,...,...,...
95,5500000000,300,458,6.0,4,0,Jawa Timur,Surabaya,Gayungan,0
96,3300000000,504,391,5.0,5,0,Jawa Timur,Surabaya,Kenjeran,0
97,235000000,36,60,2.0,1,0,Jawa Timur,Malang,Wagir,0
98,1900000000,155,82,4.0,4,0,Jawa Timur,Gresik,Manyar,0


In [22]:
prov = dataset[dataset['province'] == 'Sulawesi Selatan']
prov['city'].value_counts()

Series([], Name: count, dtype: int64)

In [23]:
data_train = dataset.sample(frac=0.8, random_state=110)
data_unseen = dataset.drop(data_train.index)

data_train.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for train: ' + str(data_train.shape))
print('Data for test: ' +str(data_unseen.shape))

Data for train: (28000, 10)
Data for test: (7000, 10)


In [24]:
cat_features = ['province', 'city']
exp = setup(data_train, target = 'price', categorical_features=cat_features, 
            ignore_features = ['z', 'district', 'latitude', 'longitude'], 
            numeric_imputation = 'mean', categorical_imputation= 'mode', 
            remove_outliers= True, normalize = True)

,Description,Value
0,Session id,6869
1,Target,price
2,Target type,Regression
3,Original data shape,"(28000, 10)"
4,Transformed data shape,"(27020, 9)"
5,Transformed train set shape,"(18620, 9)"
6,Transformed test set shape,"(8400, 9)"
7,Ignore features,4
8,Numeric features,6
9,Categorical features,2


In [25]:
training_data = get_config(variable='X_train')
training_data

,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,cluster
26745,288,500,8.0,5,0,Jawa Timur,Surabaya,0
16365,36,72,2.0,1,0,Jawa Timur,Gresik,0
8796,70,84,2.0,2,0,Jawa Timur,Batu,0
6937,280,135,5.0,3,0,Jawa Timur,Malang,0
12217,225,200,3.0,3,0,Jawa Timur,Surabaya,0
...,...,...,...,...,...,...,...,...
20550,105,105,3.0,3,0,Jawa Timur,Surabaya,0
15680,700,500,6.0,5,4,Jawa Timur,Surabaya,0
15948,140,150,4.0,2,0,Jawa Timur,Surabaya,0
20408,180,286,4.0,3,0,Jawa Timur,Malang,0


In [26]:
best_model = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,19:51:50
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,1966463142.9352,460104873227237851136.0000,15904579344.1627,0.0817,0.8995,3.1763,0.3930
br,Bayesian Ridge,2667255498.3412,433770793739091705856.0000,15852643880.6631,-0.0012,1.2330,6.4626,0.1320
dummy,Dummy Regressor,2667255498.3411,433770793739091640320.0000,15852643880.6630,-0.0012,1.2330,6.4626,0.2910
lightgbm,Light Gradient Boosting Machine,2283127604.0968,448390272852438024192.0000,16645937905.6544,-0.2789,0.9563,17.9362,0.3480
par,Passive Aggressive Regressor,3171835001.6415,449139142910087921664.0000,16695230621.2659,-0.3708,4.6246,1.0169,0.4520
knn,K Neighbors Regressor,1724201888.5806,481637843834011254784.0000,17987794374.4866,-1.0285,0.5587,1.5006,0.1880
rf,Random Forest Regressor,1872497808.8755,614496322891983355904.0000,20575449585.0642,-1.4109,0.5718,3.1074,0.9580
et,Extra Trees Regressor,1759780429.1714,612801316365596491776.0000,20387829226.8842,-1.8084,0.5612,1.5466,1.0530
xgboost,Extreme Gradient Boosting,2271035407.9847,648107387137086980096.0000,21770702470.9038,-2.4198,0.7949,2.4721,0.2310
catboost,CatBoost Regressor,2059646657.8025,704965678718589599744.0000,22703040307.1821,-3.2607,0.7432,2.4719,2.0420


In [ ]:
#evaluate precition recall curve 
evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [28]:
predict_model(best_model, data = data_unseen)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,4447574326.1817,27117429200571278032896.0000,164673705249.4152,-0.0002,0.8602,4.2215


,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,district,cluster,price,prediction_label
0,250,525,7.0,3,0,Jawa Timur,Surabaya,Dukuh Pakis,0,6800000000,6.849927e+09
1,355,200,6.0,4,0,Jawa Timur,Surabaya,Mulyorejo,0,3000000000,4.097686e+09
2,100,132,4.0,2,0,Jawa Timur,Sidoarjo,Sidoarjo,0,750000000,2.589866e+09
3,560,330,8.0,5,0,Jawa Timur,Surabaya,Rungkut,0,4500000000,7.722672e+09
4,300,240,4.0,4,0,Jawa Timur,Surabaya,Mulyorejo,0,4800000000,4.772689e+09
...,...,...,...,...,...,...,...,...,...,...,...
6995,119,119,3.0,2,0,Jawa Timur,Surabaya,Mulyorejo,0,1600000000,1.614280e+09
6996,250,180,4.0,3,0,Jawa Timur,Surabaya,Mulyorejo,0,4000000000,4.283285e+09
6997,400,240,4.0,4,0,Jawa Timur,Surabaya,Mulyorejo,0,7500000000,5.343883e+09
6998,258,150,4.0,4,0,Jawa Timur,Surabaya,Sambikerep,0,3649999999,3.386352e+09


In [29]:
finalize_model(best_model)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['buildingSize', 'landSize',
                                             'bedRooms', 'bathRooms', 'garages',
                                             'cluster'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['province', 'city'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 TransformerWr...
                                                              use_cat_names=True))),
                ('rest_encoding',
                 TransformerWrapper(include=['city'],
                                    transformer=TargetEncoder(cols=['city'],
                                                              handle_missing='return_nan'))),
                ('remove_outliers',
                 TransformerWrapper(transformer=RemoveOutliers(random_state=6869))),
                ('normalize', TransformerWrapper(transformer=StandardScaler())),
                ('actual_estimator',
                 GradientBoostingRegressor(random_state=6869))])

In [30]:
save_model(best_model, 'property_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['buildingSize', 'landSize',
                                              'bedRooms', 'bathRooms', 'garages',
                                              'cluster'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['province', 'city'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('onehot_encoding',
                  TransformerWr...
                                                               use_cat_names=True))),
                 ('rest_encoding',
                  TransformerWrapper(include=['city'],
                                     transformer=TargetEncoder(cols=['city'],
                                                               handle_missing='return_nan'))),
                 (